In [ ]:
import json

In [ ]:
import os

os.environ["GROQ_API_KEY"] = "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"

from groq import Groq

client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

In [ ]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [ ]:
documents[0]

In [ ]:
from elasticsearch import Elasticsearch

In [ ]:
es_client = Elasticsearch('http://localhost:9200')

In [ ]:
es_client.info()

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name,body=index_settings)

In [ ]:
from tqdm.auto import tqdm

In [ ]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

In [ ]:
query = 'How do I execute a command in a running docker container?'

In [ ]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name,body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [ ]:
def llm(prompt):
    response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="llama3-70b-8192",
    )
    return response.choices[0].message.content

In [ ]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [ ]:
def build_prompt(query, search_results):
    context_template = """
    Q: {question}
    A: {text}
    """.strip()
    
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT:
    {context}
    """.strip()